#環境安裝 與 資料下載

In [1]:
!wget https://raw.githubusercontent.com/Franck-Dernoncourt/pubmed-rct/master/PubMed_20k_RCT/train.txt
# !wget https://www.dropbox.com/s/yh0jzpjwh5857yj/train.txt?dl=0 

--2022-03-26 13:53:25--  https://raw.githubusercontent.com/Franck-Dernoncourt/pubmed-rct/master/PubMed_20k_RCT/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29384101 (28M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]  28.02M   105MB/s    in 0.3s    

2022-03-26 13:53:25 (105 MB/s) - ‘train.txt’ saved [29384101/29384101]



In [2]:
ls

sample_data/  train.txt


In [3]:
!head -n 20 train.txt

###24293578
OBJECTIVE	To investigate the efficacy of 6 weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at 12 weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .
METHODS	A total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .
METHODS	Outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .
METHODS	Pain was assessed using the visual analog pain scale ( 0-100 mm ) .
METHODS	Secondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and 6-min walk distance ( 6MWD ) .
METHODS	Serum levels of interleukin 1 ( IL-1 ) , IL-6 , tumor necrosis factor ( TNF ) - , and high-sensitivity C-reactive protein

In [4]:
# 環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0 && \
bash init_env.sh

--2022-03-26 13:53:25--  https://www.dropbox.com/s/cce2nnozquh8twz/init_env%20spark%203.0.1.sh?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh [following]
--2022-03-26 13:53:25--  https://www.dropbox.com/s/raw/cce2nnozquh8twz/init_env%20spark%203.0.1.sh
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0d9bbbbd5309753fadfecb1800.dl.dropboxusercontent.com/cd/0/inline/BiPr6UfCGGdFISBiI4UjOnFU8910x-Yp03FpB_AZqLUDZFrC99f4nnyPGWjaXdFiPbED8TE1bv3tPPiS3dsu0g0U9AJRNrMtijVqxzRPtAIEJ_DRzihwyOBQUTsM9PrrS2WlwuJ2hDqP7JGrxx58PX5QTAvpWEuupOsoT2QGcWtXpQ/file# [following]
--2022-03-26 13:53:26--  https://uc0d9bbbbd5309753fadfecb1800.dl.dropboxusercontent.com/cd/0/inline/BiPr6UfCGGdFISBiI4

In [5]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.9-src.zip")
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

In [6]:
pubmed = sc.textFile("./train.txt")

#資料觀察

In [7]:
pubmed = sc.textFile("./train.txt")
pubmed.count()

210040

In [8]:
pubmed.first()

'###24293578'

In [9]:
pubmed.take(20)

['###24293578',
 'OBJECTIVE\tTo investigate the efficacy of 6 weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at 12 weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .',
 'METHODS\tA total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .',
 'METHODS\tPain was assessed using the visual analog pain scale ( 0-100 mm ) .',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and 6-min walk distance ( 6MWD ) .',
 'METHODS\tSerum levels of interleukin 1 ( IL-1 ) , IL-6 , tumor necrosis factor ( TNF ) - , and hig

In [10]:
print("第13筆",pubmed.take(20)[13])
print("第14筆",pubmed.take(20)[14])
print("第15筆",pubmed.take(20)[15].split('\t'))

第13筆 
第14筆 ###24854809
第15筆 ['BACKGROUND', 'Emotional eating is associated with overeating and the development of obesity .']


In [11]:
def separate(content):
  try:
    label,sentence = content.split('\t')
    return label,sentence
  except:
    return "None"

In [12]:
Real_Content = pubmed.map(lambda x : separate(x)).filter(lambda x : x != "None")

比清理前 少了3萬筆

In [13]:
Real_Content.count()

180040

觀察處理過後的20筆

In [14]:
Real_Content.take(20)

[('OBJECTIVE',
  'To investigate the efficacy of 6 weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at 12 weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .'),
 ('METHODS',
  'A total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .'),
 ('METHODS',
  'Outcome measures included pain reduction and improvement in function scores and systemic inflammation markers .'),
 ('METHODS',
  'Pain was assessed using the visual analog pain scale ( 0-100 mm ) .'),
 ('METHODS',
  'Secondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and 6-min walk distance ( 6MWD ) .'),
 ('METHODS',
  'Serum levels of interleukin 1 ( IL-1 ) , IL-6 , tumor necrosis factor 

In [15]:
!pip install -q transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'what is the better therapy for HIV?',
    'context': 'In patients with advanced HIV disease , zidovudine appears to be more effective than didanosine as initial therapy ; however , some patients with advanced HIV disease may benefit from a change to didanosine therapy after as little as 8 to 16 weeks of therapy with zidovudine '
}
res = nlp(QA_input)
ans = res['answer']
# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 6.5 MB 45.4 MB/s 


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [16]:
ans

'zidovudine'

In [ ]:
# context = pubmed.filter(lambda x: "cardiovascular" in x and "therapy" in x).collect()
# context = pubmed.filter(lambda x: "cardiovascular" in x and "cause" in x).collect()
# context = pubmed.filter(lambda x: "caffeine" in x and "effect" in x).collect()
# context = pubmed.filter(lambda x: "antiretroviral" in x and "risk" in x).collect()
# context = pubmed.filter(lambda x: "antiretroviral" in x).collect()

In [17]:
import json
context = Real_Content.filter(lambda x: "HIV" in x[1] and "therapy" in x[1]).collect()
answers = []
paper = []

for c in context:
  label = c[0]
  sentence = c[1] 
  QA_input={'question': 'what is the therapy for the HIV?', 'context':c[1]}
  tokens = nlp(QA_input)["answer"]
  paper.append({
      "label" : label,\
      "sentence" : sentence,\
      "tokens"  : tokens,\
  })
  answers.append([c[0],nlp(QA_input)["answer"]])
  print(c[0],nlp(QA_input)["answer"])

OBJECTIVE combination antiretroviral therapy
BACKGROUND lopinavir/ritonavir
METHODS stable antiretroviral
CONCLUSIONS statin therapy
METHODS ritonavir-boosted lopinavir
CONCLUSIONS lopinavir-based therapy
METHODS antiretroviral therapy
OBJECTIVE isoniazid preventive therapy
METHODS combination antiretroviral
OBJECTIVE highly active antiretroviral therapy
METHODS statin therapy
METHODS antiretroviral
METHODS antiretroviral therapy
BACKGROUND antiretroviral
BACKGROUND boosted protease inhibitor therapy
BACKGROUND antiretroviral
CONCLUSIONS antiretroviral
BACKGROUND rifabutin
BACKGROUND antiretroviral
OBJECTIVE antiretroviral therapy
BACKGROUND molecularly targeted therapy
BACKGROUND antiretroviral
OBJECTIVE antiretroviral therapy
BACKGROUND antiretroviral
METHODS 3-drug ( N = 14 ) or 5-drug
BACKGROUND antiretroviral
OBJECTIVE rational-emotive-behavior-based therapy
METHODS antiretroviral
BACKGROUND antiretroviral
CONCLUSIONS antiretroviral
CONCLUSIONS Atorvastatin
OBJECTIVE antiretrovira

In [ ]:
print(len(paper))

116


In [ ]:
Q = 'what is the therapy for the HIV?'

In [ ]:
paper[0]['abstract']

'OBJECTIVE'

#建立Neo4j Graph

In [ ]:
!pip install neo4j

     |████████████████████████████████| 89 kB 7.2 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.1-py3-none-any.whl size=114783 sha256=2b5a86e1b90ce8fb3269820a702294f0f9984fe8952df5b9ba471a6e6977e3ef
  Stored in directory: /root/.cache/pip/wheels/fd/15/02/8379f87426cd1b74fd5891f49df9fb978423feb97650cd639b
Successfully built neo4j


連接Neo4j 雲端資料庫 https://neo4j.com/cloud/aura/

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
uri = "neo4j+s://0e9e4549.databases.neo4j.io"
user = "neo4j"
password = "tx6zEpO4cG-gs5LRohqstzUI1FWNWD-zDd7foAaaeI4"
driver = GraphDatabase.driver(uri,auth=(user, password))
def neo4j_query(query, params=None):
  with driver.session() as session:
    result = session.run(query, params)
    return pd.DataFrame([r.values() for r in result],columns=result.keys())

In [ ]:
#create Question
neo4j_query("""
UNWIND $data as item
MERGE (a:Question {id:item})
SET a.text = item
RETURN count(a)
""",{"data":Q})

,count(a)
0,1


In [ ]:
#create Answer
neo4j_query("""
UNWIND $data as item
MERGE (a:Answer {id:item["tokens"]})
SET a.label = item["label"]
SET a.content = item["sentence"]
RETURN count(a)
""",{"data":paper})

,count(a)
0,116


In [ ]:
#create types
types = ['OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS', 'BACKGROUND']
neo4j_query("""
UNWIND $data as item
MERGE (a:LABEL {id:item})
SET a.label = item
RETURN count(a)
""",{"data":types})

,count(a)
0,5


建立關係
有相同的Label 相連

In [ ]:
# Match sentence with their abstract
neo4j_query("""
MATCH (a:Answer)
WITH a
UNWIND a.label as type
MATCH (types:LABEL) where types.label = type
MERGE (a)-[:屬於]->(types)
""")

""


建立Question 與 Answer 的關係

In [ ]:
# Match Question with Answer
neo4j_query("""
MATCH (q:Question)
WITH q
MATCH (ans:Answer)
MERGE (q)-[:答案為]->(ans)
""")

""


In [ ]:
sc.parallelize(answers).map(lambda x:(x[1],1)).reduceByKey(lambda x,y:x+y).sortBy(lambda x:x[1]).collect()

[('lopinavir/ritonavir', 1),
 ('stable antiretroviral', 1),
 ('ritonavir-boosted lopinavir', 1),
 ('highly active antiretroviral therapy', 1),
 ('molecularly targeted therapy', 1),
 ('Atorvastatin', 1),
 ('atazanavir/ritonavir', 1),
 ('tenofovir alafenamide', 1),
 ('ezetimibe + fenofibrate', 1),
 ('stable PIs therapy', 1),
 ('Dialectical therapy', 1),
 ('suppressive antiretroviral therapy', 1),
 ('second-line', 1),
 ('protease inhibitor', 1),
 ('Combination antiretroviral therapy', 1),
 ('ART', 1),
 ('cytotoxic therapy', 1),
 ('PEG-IFN - / ribavirin', 1),
 ('first-line therapy', 1),
 ('tuberculosis', 1),
 ('combined antiretroviral therapy', 1),
 ('lopinavir-based therapy', 1),
 ('boosted protease inhibitor therapy', 1),
 ('3-drug ( N = 14 ) or 5-drug', 1),
 ('rational-emotive-behavior-based therapy', 1),
 ('Tenofovir alafenamide', 1),
 ('pre-antiretroviral therapy', 1),
 ('antituberculosis', 1),
 ('PI therapy', 1),
 ('zidovudine-based antiretroviral therapy', 1),
 ('reverse transcripta